In [3]:
import numpy as np
from experiments.utils.SimpleComparison import SimpleComparison
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from experiments.utils.get_kmeans_f1 import get_kmeans_f1
from experiments.utils.get_f1_for_best_k_with_knn import get_f1_for_best_k_with_knn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle
from scipy import stats



In [2]:

#
# path_to_pickle = (Path(output_dir).joinpath(Path(f"layouts.pickle"))).resolve()
# with open(path_to_pickle, 'rb') as file:
#     layouts = pickle.load(file)
# print(layouts)


# for name, dict in layouts.items():
#     print(name)
#     for type, ls in dict.items():
#         print(type)
#         for l in ls:
#             show_layout(l, use_labels=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Owner\\Desktop\\2095736w\\experiments\\sned_vs_96\\out\\layouts.pickle'

In [1]:
# for name, dict in layouts.items():
#     print(name)
#     for type, ls in dict.items():
#         print(type)
#         for l in ls:
#             print(l.iteration_number)

NameError: name 'layouts' is not defined

In [ ]:
dataset_names = [ 'coli20', 'globe', 'fashion mnist', 'mnist']
output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/sned_vs_96/out/"))).resolve().absolute()

for dataset_name in dataset_names:
    path_to_pickle = (Path(output_dir).joinpath(Path(f"layouts_{dataset_name}.pickle"))).resolve()
    with open(path_to_pickle, 'rb') as file:
        layouts = pickle.load(file)

    results_arr = np.zeros((2,15,4))
    print(dataset_name)
    for j, version in enumerate(["squad", "96"]):
        for i,layout in enumerate(layouts[version]):
            print(f"Layout: {i}")
            stress = layout.get_final_stress()
            kmeans_f1 = get_kmeans_f1(layout.get_final_positions(), layout.labels)
            k, f1_for_best_k_with_knn =  get_f1_for_best_k_with_knn(lower_bound=1, upper_bound=30,
                                                                cross_validation_folds=10,
                                                                 data=layout.data,
                                                                 labels=layout.labels)

            results_arr[j,i,0] = stress
            results_arr[j,i,1] = kmeans_f1
            results_arr[j,i,2],results_arr[j,i,3] =k,  f1_for_best_k_with_knn

    print(results_arr)
    output_dir= (Path(PROJECT_ROOT).joinpath(
        Path(f"experiments/sned_vs_96/out/"))).resolve().absolute()

    path_to_pickle_arr = (Path(output_dir).joinpath(Path(f"results_arr_{dataset_name}.pickle"))).resolve()
    with open(path_to_pickle_arr, 'wb') as pickle_out:
        pickle.dump(results_arr, pickle_out)

coli20
Layout: 0

 Computing vectorised euclidean stress 



C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Layout: 1

 Computing vectorised euclidean stress 



C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Layout: 2

 Computing vectorised euclidean stress 



...

[[[0.94695128 0.81421363 1.         0.95138933]
  [0.94701446 0.81095339 1.         0.95138933]
  [0.94707266 0.81181944 1.         0.95138933]
  [0.94703119 0.81190137 1.         0.95138933]
  [0.94703952 0.81515239 1.         0.95138933]
  [0.94702538 0.81342665 1.         0.95138933]
  [0.94699361 0.81511324 1.         0.95138933]
  [0.94690428 0.81353805 1.         0.95138933]
  [0.94703418 0.81311724 1.         0.95138933]
  [0.94696558 0.81257169 1.         0.95138933]
  [0.94703308 0.81406071 1.         0.95138933]
  [0.94703673 0.81387834 1.         0.95138933]
  [0.94711004 0.8140486  1.         0.95138933]
  [0.94704731 0.81390726 1.         0.95138933]
  [0.94703373 0.81250468 1.         0.95138933]]

 [[0.12690193 0.80679627 1.         0.95138933]
  [0.12697626 0.80701233 1.         0.95138933]
  [0.12697302 0.80714863 1.         0.95138933]
  [0.1270086  0.80403663 1.         0.95138933]
  [0.12698528 0.80459687 1.         0.95138933]
  [0.12700876 0.80446413 1.         0.

C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Layout: 1

Not enough memory to allocate for a numpy array for stress calculation. 
Stress will be calculated with a Python loop

 Computing un-vectorised euclidean stress 



C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Layout: 2

Not enough memory to allocate for a numpy array for stress calculation. 
Stress will be calculated with a Python loop

 Computing un-vectorised euclidean stress 

